# Setup Data

## Import libraries

In [2]:
import boto3
import os
import pandas as pd
import sagemaker

from IPython.core.display import display, HTML
from pyathena import connect

/tmp/ipykernel_2009/4216669510.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
# check dependencies are stored
%store

Stored variables and their in-db values:
setup_dependencies_passed             -> True
setup_s3_bucket_passed                -> True


## Setup S3 Bucket

In [5]:
# save Amazon information
account_id = boto3.client("sts").get_caller_identity().get("Account")
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
session = sagemaker.Session()

#bucket = session.default_bucket()
#sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [7]:
print(region)

us-east-1


In [9]:
# create a new S3 bucket
s3 = boto3.client('s3')
bucket_name = 'ai-cooking-assistant-5432'
s3.create_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': '18WR8N2J6SX1EBA9',
  'HostId': 'isuk4guujxV5SgVZQNjGleaN8Rdi4F2FuFiqdsSTg28+VINMoTCZnVbl6+KzVDFBjYJvMz4sul4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'isuk4guujxV5SgVZQNjGleaN8Rdi4F2FuFiqdsSTg28+VINMoTCZnVbl6+KzVDFBjYJvMz4sul4=',
   'x-amz-request-id': '18WR8N2J6SX1EBA9',
   'date': 'Thu, 23 May 2024 01:36:16 GMT',
   'location': '/ai-cooking-assistant-5432',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/ai-cooking-assistant-5432'}

## Copy Data to our S3 Bucket

In [14]:
# save path to wizards of tasks bucket
s3_public_path = "s3://wizard-of-tasks/"
print(s3_public_path)

s3://wizard-of-tasks/


In [15]:
# save path to S3 bucket
s3_path = "s3://{}/data/".format(bucket_name)
print(s3_path)

s3://ai-cooking-assistant-5432/data


In [19]:
# copy data from local to s3
!aws s3 cp $s3_public_path $s3_path

In [18]:
# check it uploaded
display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/sagemaker-{}-{}/?region={}&tab=overview">S3 Bucket</a></b>'.format(
            region, account_id, region
        )
    )
)

In [24]:
# List objects in the source bucket
source_bucket = "wizard-of-tasks"
response = s3.list_objects_v2(Bucket=source_bucket,)

ParamValidationError: Parameter validation failed:
Invalid bucket name "arn:aws:s3:::wizard-of-tasks": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:s3:[a-z\-0-9]+:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-]{1,63}$|^arn:(aws).*:s3-outposts:[a-z\-0-9]+:[0-9]{12}:outpost[/:][a-zA-Z0-9\-]{1,63}[/:]accesspoint[/:][a-zA-Z0-9\-]{1,63}$"

In [27]:
source_bucket = "wizard-of-tasks"
# Create Boto3 client for S3 with the specified region
s3 = boto3.client('s3', region_name='us-west-2')

# List objects in the source bucket
response = s3.list_objects_v2(Bucket=source_bucket)

print(response)

{'ResponseMetadata': {'RequestId': 'SN0FK2DT1S844DTP', 'HostId': 'L/8HghjsSC+5mn+CWayj/QJVueVd3p8v81ZVpvsRGuj4ghvosmhmIVbExIcG+w/drERP7bispHkLqeRdXzr1iA==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'L/8HghjsSC+5mn+CWayj/QJVueVd3p8v81ZVpvsRGuj4ghvosmhmIVbExIcG+w/drERP7bispHkLqeRdXzr1iA==', 'x-amz-request-id': 'SN0FK2DT1S844DTP', 'date': 'Thu, 23 May 2024 02:03:54 GMT', 'x-amz-bucket-region': 'us-west-2', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Contents': [{'Key': 'README.md', 'LastModified': datetime.datetime(2022, 10, 7, 22, 59, 54, tzinfo=tzlocal()), 'ETag': '"16a8aa13b9d476be1ddf41491896d3b7"', 'Size': 6354, 'StorageClass': 'STANDARD'}, {'Key': 'wizard_of_tasks_cooking_v1.0.json', 'LastModified': datetime.datetime(2022, 10, 7, 23, 1, 45, tzinfo=tzlocal()), 'ETag': '"09619ab31682ca2235992f26fd986235"', 'Size': 3826844, 'StorageClass': 'INTELLIGENT_TIERING'}, {'Key': 'wizard_of_tas

## Setup table in Athena

In [10]:
# set database name
database_name = "awsdata"

# set S3 staging directory
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

# create connection
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# create database if it doesn't exist
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
pd.read_sql(statement, conn)

/tmp/ipykernel_1093/2473272394.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [11]:
# verify the database has been created
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_1093/2543210509.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,awsdata
1,default


In [12]:
# set table name
table_name_csv = "dataset_csv"

# create table
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         row_id int,
         track_id string,
         artists string,
         track_name string,
         popularity int,
         duration_ms int,
         explicit boolean,
         danceability float,
         energy float,
         key int,
         loudness float,
         mode int,
         speechiness float,
         acousticness float,
         instrumentalness float,
         liveness float,
         valence float,
         tempo float,
         time_signature int,
         track_genre string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\\t' LINES TERMINATED BY '\\n' LOCATION '{}/'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_path
)

pd.read_sql(statement, conn)

/tmp/ipykernel_1093/1404487769.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [13]:
# verify table has been created
statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_1093/1902741135.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,dataset_csv


In [14]:
# pull data
statement = """SELECT * FROM {}.{}""".format(
    database_name, table_name_csv
)
df = pd.read_sql(statement, conn)
df.head(5)

/tmp/ipykernel_1093/3926281859.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,row_id,track_id,artists,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [15]:
# review the new table in Athena
display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">Data Catalog</a></b>'.format(
            region
        )
    )
)

## Queries

### Query 1
List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99

In [16]:
# pandas
# filter where popularity is greater than or equal to 99
df1a = df[df['popularity'] >= 99]

# select the artist, track_name, and popularity columns
df1a = df1a[['artists', 'track_name', 'popularity']]

# Display the resulting DataFrame
print(df1a)

                    artists                             track_name  popularity
20001  Sam Smith;Kim Petras              Unholy (feat. Kim Petras)         100
51664      Bizarrap;Quevedo  Quevedo: Bzrp Music Sessions, Vol. 52          99
81051  Sam Smith;Kim Petras              Unholy (feat. Kim Petras)         100


In [17]:
# sql
statement = """
    SELECT 
        artists, 
        track_name,
        popularity
    FROM {}.{}
    WHERE popularity >= 99
    """.format(
    database_name, table_name_csv
)
df1b = pd.read_sql(statement, conn)
print(df1b)

/tmp/ipykernel_1093/1127352022.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1b = pd.read_sql(statement, conn)


                artists                             track_name  popularity
0  Sam Smith;Kim Petras              Unholy (feat. Kim Petras)         100
1      Bizarrap;Quevedo  Quevedo: Bzrp Music Sessions, Vol. 52          99
2  Sam Smith;Kim Petras              Unholy (feat. Kim Petras)         100


## Query 2
List artists with an average popularity of 92

In [18]:
# pandas
# group the DataFrame by 'artist' and calculate the average popularity
df2a = df.groupby('artists')['popularity'].mean()

# filter the artists based on average popularity being equal to 92
df2a = df2a[df2a == 92]
print(df2a)

artists
Harry Styles         92.0
Rema;Selena Gomez    92.0
Name: popularity, dtype: float64


In [19]:
# sql
statement = """
    SELECT 
        artists, 
        AVG(popularity) 
    FROM {}.{}
    GROUP BY artists
    HAVING AVG(popularity) = 92;
""".format(database_name, table_name_csv)
df2b = pd.read_sql(statement, conn)
print(df2b)

/tmp/ipykernel_1093/300158246.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2b = pd.read_sql(statement, conn)


             artists  _col1
0  Rema;Selena Gomez   92.0
1       Harry Styles   92.0


## Query 3
List the Top 10 most energetic genres

In [20]:
# pandas
# group the DataFrame by 'genre' and calculate the mean energy
df3a = df.groupby('track_genre')['energy'].mean()

# sort the genres based on mean energy values in descending order
df3a = df3a.sort_values(ascending=False)

# select the top 10 most energetic genres
df3a = df3a.head(10)
print(df3a)

track_genre
death-metal      0.931470
grindcore        0.924201
metalcore        0.914485
happy            0.910971
hardstyle        0.901246
drum-and-bass    0.876635
black-metal      0.874897
heavy-metal      0.874003
party            0.871237
j-idol           0.868677
Name: energy, dtype: float64


In [21]:
# sql 
statement = """
    SELECT 
        track_genre,
        AVG(energy) AS avg_energy
    FROM {}.{}
    GROUP BY track_genre
    ORDER BY avg_energy DESC
    LIMIT 10;
""".format(database_name, table_name_csv)
df3b = pd.read_sql(statement, conn)
print(df3b)

/tmp/ipykernel_1093/632945844.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df3b = pd.read_sql(statement, conn)


     track_genre  avg_energy
0    death-metal    0.931470
1      grindcore    0.924201
2      metalcore    0.914485
3          happy    0.910971
4      hardstyle    0.901246
5  drum-and-bass    0.876635
6    black-metal    0.874897
7    heavy-metal    0.874003
8          party    0.871237
9         j-idol    0.868677


### Query 4
How many tracks is Bad Bunny on?

In [22]:
# pandas
# filter to find Bad Bunny in 'artists' column
df4a = df[df['artists'].str.contains('Bad Bunny', case=False)]

# count the number of rows
print(df4a.shape[0])

416


In [23]:
# sql 
statement = """
    SELECT 
        COUNT(*) AS num_tracks
    FROM {}.{}
    WHERE artists LIKE '%Bad Bunny%'
""".format(database_name, table_name_csv)
df4b = pd.read_sql(statement, conn)
print(df4b)

/tmp/ipykernel_1093/2534342410.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df4b = pd.read_sql(statement, conn)


   num_tracks
0         416


### Query 5
Show the top 10 genres in terms of popularity sorted by their most popular track

In [24]:
# pandas
# calculate max popularity by 'track_genre'
df5a = df.groupby('track_genre')['popularity'].max()

# sort the genres based on maximum popularity in descending order
df5a = df5a.sort_values(ascending=False)
print(df5a.head(10))

track_genre
pop          100
dance        100
hip-hop       99
latino        98
reggaeton     98
edm           98
latin         98
reggae        98
piano         96
rock          96
Name: popularity, dtype: int64


In [25]:
# sql
statement = """
    SELECT 
        track_genre, 
        MAX(popularity) AS max_popularity
    FROM {}.{}
    GROUP BY track_genre
    ORDER BY max_popularity DESC
    LIMIT 10;
""".format(database_name, table_name_csv)
df5b = pd.read_sql(statement, conn)
print(df5b)

/tmp/ipykernel_1093/3371284048.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df5b = pd.read_sql(statement, conn)


  track_genre  max_popularity
0       dance             100
1         pop             100
2     hip-hop              99
3       latin              98
4      reggae              98
5   reggaeton              98
6         edm              98
7      latino              98
8        rock              96
9       piano              96


## Shut down notebook resources

In [26]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [27]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>